In [1]:
import numpy as np
import os
import moleculegraph

from tools.utils import work_json, get_system_settings
from tools.prepare_lammps import LAMMPS_input

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [10]:
#### Name of the mixture everything will be created in this subfolder ####

Gruppe   = "Mixtures"

name_mix = "mie1_mie2"

volatile_component = "mie1"

name1   = "mie1"
name2   = "mie2"

molecule_graphstrings = {
                            "mie1": "[MIE1]",
                            "mie2": "[MIE2]"
}

## Define unqiue key for mixture (either constant temperature (then give temp in K) or constant pressure (then give press in bar)) ##

dict_key = "100_4"

#### Specify temperatures [K], pressures [bar] and mass densities [kg/m^3] ####

ref_dict  = work_json("reference_dict.json")

temp_all  = ref_dict[name_mix][dict_key]["temp"]
pres_all  = ref_dict[name_mix][dict_key]["system_building"]["pressure"]
dens_all  = ref_dict[name_mix][dict_key]["system_building"]["density"]

#### Specify free energy intermediates 

## Method ##

method = "TI"

# Lennard-Jones simulation
txt_sim,sim_lj = "sim_lj",True

# Coulomb simulation
#txt_sim,sim_lj = "sim_coul",False

# If system should be build with PACKMOL (pure 1, insertion 1 in mixture, insertion 2 in mixture, pure 2)
system_build = [True, True, True, True]

## Intermediates ##

# Van der Waals
l_lj   = [ 0.0, 0.1, 0.13, 0.17, 0.21, 0.25, 0.3, 0.35, 0.45, 0.5, 0.55, 0.7, 0.9, 1.0 ]
#l_lj  = [ 0.0, 0.1, 0.17, 0.21, 0.26, 0.3, 0.38, 0.45, 0.7, 0.9, 1.0 ]
#l_lj  = [ 0.0, 0.1, 0.17, 0.21, 0.26, 0.3, 0.45, 0.7, 1.0 ]
#l_lj  = [ 0.  , 0.21, 0.3 , 0.45, 0.7 , 1.  ]

# Coulomb
l_q    = [ 0.002, 0.25, 0.5, 0.75, 1.0 ]

## Differentiate between van der Waals and Coulomb portion of free solvation energy ##

if sim_lj:
    lambda_lj_sim  = l_lj
    lambda_q_sim   = np.ones(len(l_lj))*10**-9
    print("Lambda van der Waals:\n",lambda_lj_sim)
else:
    lambda_lj_sim = np.ones(len(l_q))
    lambda_q_sim  = l_q
    print("Lambda Coulomb:\n",lambda_q_sim)

Lambda van der Waals:
 [0.0, 0.1, 0.13, 0.17, 0.21, 0.25, 0.3, 0.35, 0.45, 0.5, 0.55, 0.7, 0.9, 1.0]


In [12]:
# Build system and write LAMMPS input file for whole mixture

for k,(nam1,nam2) in enumerate( [ [name1, name1], [name1, name2], [name2, name1], [name2,name2] ] ):

    print("\nMixture of %s and %s\n"%(nam1,nam2))

    # Define used molecule
    molecule1 = moleculegraph.molecule(molecule_graphstrings[nam1])
    molecule2 = moleculegraph.molecule(molecule_graphstrings[nam2])

    # Differentiate if pure components are simulated or mixtures from x1=0.0...0.9
    temperatures,pressures,densities  = get_system_settings(nam1,nam2,volatile_component,temp_all,pres_all,dens_all)

    print("Temperatures specifided:\n",temperatures)
    print("Pressures specified:\n",pressures)
    print("Densities specified:\n",densities)

    ntotal        = 900

    x1            = np.array( ref_dict[name_mix][dict_key]["system_building"]["x1"]  )

    nmols1        = (x1*ntotal).astype(int)
    nmols2        = ntotal-nmols1

    nmol_list_tot = [[n1,n2] for n1,n2,d in zip(nmols1,nmols2,densities)]

    print("\nMolecule numbers species 1:",np.array(nmol_list_tot)[:,0])
    print("Molecule numbers species 2:",np.array(nmol_list_tot)[:,1])

    name      = "mix_%s_%s"%(nam1,nam2)
    name_list = [nam1,nam2]
    mol_list  = [molecule1,molecule2]

    #### Write LAMMPS data file (atoms, bonds, angles, dihedrals) and playmol input ####

    maindir = os.getcwd()

    if not os.path.exists("../"+Gruppe+"/"+name_mix+"/"+dict_key+"/"+name): os.makedirs("../"+Gruppe+"/"+name_mix+"/"+dict_key+"/"+name)

    ## Initialize the LAMMPS class and create input force field file for playmol to build the system ##

    # Load in the used ff 
    ff_path = "../force-fields/forcefield_lammps_UA_TAMie.toml"
    ff_path = "../force-fields/forcefield_lammps.toml"
    ff_path = "../force-fields/forcefield_model_fluid.toml"
    # ff_path = "../force-fields/forcefield_OPLS_AA.toml"

    # Define path for playmol ff 
    playmol_ff_path = "../inout_playmol/"+name_mix+"/"+dict_key+"/"+name+"/"+name+".playmol"

    # Initialize the LAMMPS class 
    lammps_input_class = LAMMPS_input(mol_list,ff_path,playmol_ff_path)

    ## If system should be build localy with playmol while running this script ##

    build = system_build[k]

    # Loop over different mixture compositions and create input file 
    for i,(nmol_list,rho) in enumerate(zip(nmol_list_tot,densities)):
        
        # String of current composition
        txt_comp = str( np.round( x1[i] ,1) )

        # Initialize the LAMMPS data file function 
        lammps_input_class.data_file_init(nmol_list,rho,maindir)
                                        
        # If wanted build the system with playmol 
        if build: 
            playmol_path = "../inout_playmol/"+name_mix+"/"+dict_key+"/"+name+"/"+name+"_"+str(i)+".mol"
            lammps_input_class.build_playmol(playmol_path,name_list,name,i)

        # Write LAMMPS data file 
        xyz_location         = "../inout_playmol/"+name_mix+"/"+dict_key+"/"+name+"/"+name+"_"+str(i)+".xyz"
        data_output_location = "../"+Gruppe+"/"+name_mix+"/"+dict_key+"/"+name+"/"+txt_sim+"/x%s"%txt_comp+"/"+name+"_x%s.lmp"%txt_comp

        lammps_input_class.write_lammps_data(xyz_location,data_output_location)

    #### Write LAMMPS input file ####

    ### Definitons for LAMMPS simulation ###

    settings              = {}
    settings["equiltime"] = int(5*10**6)
    settings["runtime"]   = int(3*10**6)
    settings["timestep"]  = 1 #femto seconds
    settings["restart"]   = 0 #0 if no restart, 1 if restart is wanted.

    ## Define if special bonds for LAMMPS should be used ##
    settings["sb_dict"]     = {"lj":[0.0,0.0,0.0],"coul":[0.0,0.0,0.0]}

    ## Define mixing rule for FF sigma parameters (arithmetic or geometric) ##
    settings["mixing_rule"] = "arithmetic"

    ## Define the keys for the atoms, bonds or angles that should be constrained using SHAKE ( use notation as defined in ff ) ##

    s_atom  = [[]]
    s_bond  = [['OH_tip3p', 'cH_tip3p']]
    s_angle = [['cH_tip3p', 'OH_tip3p', 'cH_tip3p']]

    settings["shake"] = [s_atom,s_bond,s_angle]

    ## Define appendix to folder (either lj or coul) ##
    settings["txt_add"] = "lj" if sim_lj else "coul"

    ## Define free solvation energy portion, free energy method and lambda intermediates step ##

    settings["sim_lj"]  = sim_lj
    settings["method"]  = method
    settings["lambdas"] = [ lambda_lj_sim, lambda_q_sim ]

    ## Define name of restart file after equilibration ##

    settings["r_equil"]  = "equil.restart"

    ### Loop over all compositions to write LAMMPS input files ###

    for l,(nmol_list,d,p,t) in enumerate(zip(nmol_list_tot,densities,pressures,temperatures)):

        # String of current composition
        txt_comp = str( np.round( x1[l] ,1) ) 
        
        # Define relative path of LAMMPS data file and the output path of the input file #
        
        data_path        = "../../"+name+"_x%s.lmp"%txt_comp
        input_file_name  = "../"+Gruppe+"/"+name_mix+"/"+dict_key+"/"+name+"/"+txt_sim+"/x%s"%txt_comp+"/"+method+"/%s"%txt_sim+"_%s/"+"%s_%s.input"%(name,settings["txt_add"])

        ## Initialize LAMMPS input file function ##

        lammps_input_class.input_file_init(settings)

        ## Write LAMMPS input file ##

        lammps_input_class.write_lammps_input(t,p,nmol_list,settings,data_path,input_file_name)
        



Mixture of mie1 and mie1

Temperatures specifided:
 [100]
Pressures specified:
 [1.55]
Densities specified:
 [1355.27]

Molecule numbers species 1: [1]
Molecule numbers species 2: [899]
Playmol (Version: 04 Mar 2021)
Reading file mix_mie1_mie1_0.mol...
Adding variable rho with parameters 1.35527
Adding variable seed with parameters 950751
Adding variable nmols1 with parameters 1
Adding variable nmols2 with parameters 899
Adding variable x with parameters 0
Including file mix_mie1_mie1.playmol
Adding atom type MIE1 with parameters mie/cut 0.23646 3.4098 10.0 6.0
Adding diameter MIE1 with parameters 3.4098
Adding mass MIE1 with parameters 39.948
Adding charge MIE1 with parameters 0.0
Adding atom M1 with parameters MIE1
Adding charge M1 with parameters 0.0
Adding molecule M1 with parameters 1
Adding atom M2 with parameters MIE1
Adding charge M2 with parameters 0.0
Adding molecule M2 with parameters 2
Reading geometric data from file mie1_playmol.xyz...
Number of data to be read: 1
1 : M1